In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 23552365
paper_name = 'gonzalez_daran_2003' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
xl = pd.ExcelFile('raw_data/Butanol tolerance. Screening knockout collection.xlsx')
sheets = xl.sheet_names

In [6]:
len(sheets)


55

In [7]:
plate_list = []
for s in sheets:
    print('%s' % s)
    original_data = xl.parse(s)
    
    plate_num = int(s.split(' ')[0][2:])
    rows = np.arange(1,9)
    cols = np.arange(1,13)
        
    control_plate1 = pd.DataFrame(index=rows, columns=cols, data=original_data.iloc[5:13,2:14].values)
    test_plate1 = pd.DataFrame(index=rows, columns=cols, data=original_data.iloc[17:25,2:14].values)
    
    print(original_data.shape)
    
    if original_data.shape[1] > 14:
        control_plate2 = pd.DataFrame(index=rows, columns=cols, data=original_data.iloc[5:13,16:28].values)
        test_plate2 = pd.DataFrame(index=rows, columns=cols, data=original_data.iloc[17:25,16:28].values)
    else:
        control_plate2 = control_plate1.copy()
        test_plate2 = test_plate1.copy()
    
    control_plate1 = pd.melt(control_plate1.reset_index(), id_vars='index', var_name='col')
    control_plate1.columns = ['row','col','value']
    control_plate2 = pd.melt(control_plate2.reset_index(), id_vars='index', var_name='col')
    control_plate2.columns = ['row','col','value']
    
    control_plate = control_plate1.merge(control_plate2, on=['row','col'])
    control_plate['value'] = control_plate[['value_x','value_y']].mean(axis=1)
    
    test_plate1 = pd.melt(test_plate1.reset_index(), id_vars='index', var_name='col')
    test_plate1.columns = ['row','col','value']
    test_plate2 = pd.melt(test_plate2.reset_index(), id_vars='index', var_name='col')
    test_plate2.columns = ['row','col','value']
    
    test_plate = test_plate1.merge(test_plate2, on=['row','col'])
    test_plate['value'] = test_plate[['value_x','value_y']].mean(axis=1)
    
    plate = test_plate[['row','col','value']].merge(control_plate[['row','col','value']], on=['row','col'])
    plate['value'] = plate['value_x'] / plate['value_y']
    plate['num'] = plate_num
    
    plate_list.append(plate)

MP72
(49, 56)
MP71
(49, 56)
MP70
(49, 56)
MP51
(49, 56)
MP50
(49, 56)
MP49
(49, 56)
MP48
(49, 56)
MP47
(49, 56)
MP46
(49, 56)
MP45
(49, 56)
MP44
(49, 56)
MP43
(49, 56)
MP42
(49, 56)
MP41
(49, 56)
MP40
(49, 56)
MP39
(49, 56)
MP38
(49, 56)
MP37
(49, 56)
MP36
(49, 56)
MP35
(49, 56)
MP34
(49, 56)
MP33
(49, 56)
MP32
(49, 56)
MP31
(49, 56)
MP30
(49, 56)
MP29
(49, 56)
MP28
(49, 56)
MP27
(49, 56)
MP26
(49, 56)
MP25
(49, 56)
MP24
(49, 56)
MP23
(49, 56)
MP22
(49, 56)
MP21
(49, 56)
MP20
(49, 56)
MP19
(49, 56)
MP18
(49, 56)
MP17
(49, 56)
MP16
(49, 56)
MP15
(49, 56)
MP14
(49, 56)
MP13
(49, 56)
MP12
(49, 56)
MP11
(49, 56)
MP10
(49, 56)
MP9
(49, 56)
MP8
(49, 56)
MP7
(49, 56)
MP6
(49, 56)
MP5
(49, 56)
MP4
(49, 56)
MP3
(49, 56)
MP2
(49, 56)
MP1 plate 2
(49, 14)
MP1 Plate 1
(49, 14)


In [8]:
original_data = pd.concat(plate_list, axis=0)

In [9]:
original_data.shape

(5280, 6)

In [10]:
# Load the platemap
plate_map = pd.read_excel('raw_data/Knockout collection map.xls', sheet_name='DATA')

In [11]:
plate_map.shape

(5007, 8)

In [12]:
plate_map = plate_map.loc[plate_map['Plate'].notnull() & plate_map['Row'].notnull() & plate_map['Col'].notnull(),:]

In [13]:
plate_map['num'] = plate_map['Plate'].astype(int)

In [14]:
plate_map['col'] = plate_map['Col'].astype(int)

In [15]:
plate_map['row'] = plate_map['Row'].apply(lambda x: 'ABCDEFGH'.find(x)+1)

In [16]:
original_data = original_data.merge(plate_map[['ORF name','num','col','row']], on=['num','row','col'])

In [17]:
original_data.head()

,row,col,value_x,value_y,value,num,ORF name
0,1,1,0.53795,0.8827,0.609437,72,YAL016C-B
1,2,1,0.53825,0.8666,0.621105,72,YBR072C-A
2,4,1,0.53325,0.8702,0.612790,72,YDR246W-A
3,5,1,0.52905,0.5387,0.982087,72,YER175W-A
4,6,1,0.53885,0.5449,0.988897,72,YGR121W-A


In [18]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 5022 x 7


In [19]:
original_data['orf'] = original_data['ORF name'].astype(str)

In [20]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [21]:
original_data.loc[original_data['orf']=='YLR287-A','orf'] = 'YLR287C-A'

In [22]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [23]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [row, col, value_x, value_y, value, num, ORF name, orf]
Index: []


In [24]:
original_data['data'] = original_data['value']

In [25]:
original_data.set_index('orf', inplace=True)

In [26]:
original_data = original_data[['data']].copy()

In [27]:
original_data = original_data.groupby(original_data.index).mean()

In [28]:
original_data.shape

(4838, 1)

# Prepare the final dataset

In [29]:
data = original_data.copy()

In [30]:
dataset_ids = [127]
datasets = datasets.reindex(index=dataset_ids)

In [31]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [32]:
data.head()

dataset_id,127
data_type,value
orf,
YAL002W,0.331092
YAL004W,0.508252
YAL005C,0.508469
YAL007C,0.539061
YAL008W,0.550875


## Subset to the genes currently in SGD

In [33]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 23


In [34]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,127
,data_type,value
gene_id,orf,
2,YAL002W,0.331092
1863,YAL004W,0.508252
4,YAL005C,0.508469
5,YAL007C,0.539061
6,YAL008W,0.550875


# Normalize

In [35]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [36]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [37]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id            127          
data_type           value    valuez
gene_id orf                        
2       YAL002W  0.331092 -0.684431
1863    YAL004W  0.508252 -0.157636
4       YAL005C  0.508469 -0.156990
5       YAL007C  0.539061 -0.066026
6       YAL008W  0.550875 -0.030896

# Print out

In [38]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [39]:
from IO.save_data_to_db3 import *

In [40]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 23552365...
Inserting the new data...


100%|██████████| 1/1 [00:07<00:00,  7.72s/it]

Updating the data_modified_on field...
